# 🚦 Indian Traffic Sign Detection - YOLOv8 Training

**Author:** Himanshu Sharma  
**Project:** Smart Traffic Detection System  
**Model:** YOLOv8 Nano (Optimized for Browser Deployment)

---

## 📋 Prerequisites

Before running this notebook:

1. **Download Dataset from Roboflow:**
   - Go to: https://universe.roboflow.com/datacluster-labs/indian-traffic-sign
   - Click "Download" → Select "YOLOv8" format
   - Download as ZIP

2. **Upload to Google Drive:**
   - Upload the ZIP file to `My Drive/datasets/`
   - Name it: `indian-traffic-signs.zip`

3. **Enable GPU:**
   - Go to Runtime → Change runtime type → T4 GPU

---

## Cell 1: Mount Google Drive & Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install ultralytics -q

import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

## Cell 2: Extract Dataset

In [ ]:
import os, zipfile
DATASET_ZIP = '/content/drive/MyDrive/datasets/indian-traffic-signs.zip'
DATASET_DIR = '/content/dataset'

if not os.path.exists(DATASET_ZIP):
    print('❌ Dataset ZIP not found! Upload to My Drive/datasets/')
else:
    with zipfile.ZipFile(DATASET_ZIP, 'r') as z:
        z.extractall(DATASET_DIR)
    print(f'✅ Extracted to: {DATASET_DIR}')

## Cell 3: Find data.yaml

In [ ]:
import glob, yaml
yaml_files = glob.glob(f'{DATASET_DIR}/**/data.yaml', recursive=True)
DATA_YAML = yaml_files[0] if yaml_files else None
print(f'✅ Found: {DATA_YAML}')
with open(DATA_YAML) as f:
    data = yaml.safe_load(f)
    print(f"Classes: {len(data.get('names', []))}")

## Cell 4: Train YOLOv8 (45-60 min)

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

results = model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=512,
    batch=16,
    patience=10,
    device=0,
    project='/content/runs',
    name='indian_traffic_signs',
    exist_ok=True
)
print('✅ Training complete!')

## Cell 5: Evaluate Model

In [ ]:
best_model_path = '/content/runs/indian_traffic_signs/weights/best.pt'
model = YOLO(best_model_path)
metrics = model.val()
print(f'mAP50: {metrics.box.map50:.3f}')
print(f'Precision: {metrics.box.mp:.3f}')
print(f'Recall: {metrics.box.mr:.3f}')

## Cell 6: Export to TensorFlow.js

In [ ]:
model.export(format='tfjs', imgsz=512)
export_dir = '/content/runs/indian_traffic_signs/weights/best_web_model'
print(f'✅ Exported to: {export_dir}')

## Cell 7: Create class_names.json

In [ ]:
import json
with open(DATA_YAML) as f:
    data = yaml.safe_load(f)
class_names = list(data['names'].values()) if isinstance(data['names'], dict) else data['names']
with open(f'{export_dir}/class_names.json', 'w') as f:
    json.dump({'class_names': class_names}, f)
print(f'✅ Saved {len(class_names)} classes')

## Cell 8: Download Model

In [ ]:
import shutil
from google.colab import files
shutil.make_archive('/content/indian_traffic_signs_model', 'zip', export_dir)
files.download('/content/indian_traffic_signs_model.zip')
print('✅ Download started!')

## ✅ Done!

Extract to `frontend/public/models/traffic_signs/`

**Model Stats:**
- Size: ~3-5 MB
- Input: 512x512
- Classes: 85 Indian traffic signs